# Eat Safe, Love

## Notebook Set Up

In [62]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [63]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [64]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [65]:
# review the collections in our database
collection_list = db.list_collection_names()
print(collection_list)

['establishments']


In [66]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [67]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

0
None


In [68]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(establishments.find(query))
                  
# Display the number of rows in the DataFrame
print(len(df))
                  
# Display the first 10 rows of the DataFrame
df.head(10)

0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [69]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex':'London'}, 'RatingValue': {'$gte': 4} }

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

0
None


In [70]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
print(len(df))

# Display the first 10 rows of the DataFrame
df.head(10)

0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [71]:
# Search within 0.01 degree on either side of the latitude and longitude.
degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

# Rating value must equal 5
query = {'RatingValue': 5, 
         'geocode.latitude': {'$lte': (latitude + degree_search), '$gte': (latitude - degree_search)}, 
         'geocode.longitude': {'$lte': (longitude + degree_search), '$gte': (longitude - degree_search)}}

# Sort by hygiene score
sort =  [('scores.Hygiene',1)]

# Store the results in a list
results = list(establishments.find(query).sort(sort).limit(5))

# Print the results
print(results)

[]


In [73]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(establishments.find(query).sort(sort).limit(5))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [74]:
# Create a pipeline that:

# 1. Matches establishments with a hygiene score of 0
qry_match = {'$match': {'scores.Hygiene': {'$eq': 0}}}

# 2. Groups the matches by Local Authority
qry_group = {'$group': {'_id': "$LocalAuthorityName", 'count': { '$sum': 1 }}}

# 3. Sorts the matches from highest to lowest
qry_sort = {'$sort': { 'count': -1 }}

# Store the queries in a pipeline
pipeline = [qry_match, qry_group, qry_sort]

# Store the pipeline in a list
result = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(len(result))

# Print the first 10 results
print(result[0:10])

0
[]


In [76]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result)
# Display the number of rows in the DataFrame
print(len(result))
# Display the first 10 rows of the DataFrame
df.head(10)

0


""
